ESTANDARIZACIÓN

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import researchpy as rp
import math

from scipy import stats
import researchpy as rp
from scipy.stats import levene
from scipy.stats import chi2_contingency
from sklearn.preprocessing import RobustScaler


plt.rcParams["figure.figsize"] = (10,8)
pd.options.display.max_columns = None

import warnings
warnings.filterwarnings('ignore')

In [16]:
df = pd.read_pickle('../data/bikes_limpieza_sin_totales.pkl')
df.head(2)

,fecha,estacion,año,mes,festivo,dia,laborable,meteo,temperatura,sensacion_termica,humedad,velo_viento,usu_casuales,usu_registrados
0,2018-01-01,invierno,2018,ene,festivo,lun,no laborable,nublado,14.110847,18.18125,80.5833,10.749882,331,654
1,2018-01-02,invierno,2018,ene,no festivo,mar,laborable,nublado,14.902598,17.68695,69.6087,16.652113,131,670


Vamos a eliminar la VP numérica "temperatura" que vimos en visualización que correlacionaba muy alto (0.99) con sensación térmica.
Vamos a eliminar de nuestro modelo predictivo la V "usu_casuales" porque aquí no aplica como VP.

In [17]:
# Vamos a probar a hacer una copia sin eliminar la sensación térmica:
df_temp_sens = df.copy()
df_temp_sens

,fecha,estacion,año,mes,festivo,dia,laborable,meteo,temperatura,sensacion_termica,humedad,velo_viento,usu_casuales,usu_registrados
0,2018-01-01,invierno,2018,ene,festivo,lun,no laborable,nublado,14.110847,18.18125,80.5833,10.749882,331,654
1,2018-01-02,invierno,2018,ene,no festivo,mar,laborable,nublado,14.902598,17.68695,69.6087,16.652113,131,670
2,2018-01-03,invierno,2018,ene,no festivo,mie,laborable,buen tiempo,8.050924,9.47025,43.7273,16.636703,120,1229
3,2018-01-04,invierno,2018,ene,no festivo,jue,laborable,buen tiempo,8.200000,10.60610,59.0435,10.739832,108,1454
4,2018-01-05,invierno,2018,ene,no festivo,vie,laborable,buen tiempo,9.305237,11.46350,43.6957,12.522300,82,1518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,2019-12-27,invierno,2019,dic,no festivo,vie,laborable,nublado,10.420847,11.33210,65.2917,23.458911,247,1867
726,2019-12-28,invierno,2019,dic,no festivo,sab,no laborable,nublado,10.386653,12.75230,59.0000,10.416557,644,2451
727,2019-12-29,invierno,2019,dic,no festivo,dom,no laborable,nublado,10.386653,12.12000,75.2917,8.333661,159,1182
728,2019-12-30,invierno,2019,dic,no festivo,lun,laborable,buen tiempo,10.489153,11.58500,48.3333,23.500518,364,1432


In [18]:
df.drop(["sensacion_termica", "usu_casuales"], axis= 1, inplace= True)

In [19]:
df_temp_sens.drop("usu_casuales", axis= 1, inplace= True)

In [7]:
df.head()

,fecha,estacion,año,mes,festivo,dia,laborable,meteo,temperatura,humedad,velo_viento,usu_registrados
0,2018-01-01,invierno,2018,ene,festivo,lun,no laborable,nublado,14.110847,80.5833,10.749882,654
1,2018-01-02,invierno,2018,ene,no festivo,mar,laborable,nublado,14.902598,69.6087,16.652113,670
2,2018-01-03,invierno,2018,ene,no festivo,mie,laborable,buen tiempo,8.050924,43.7273,16.636703,1229
3,2018-01-04,invierno,2018,ene,no festivo,jue,laborable,buen tiempo,8.200000,59.0435,10.739832,1454
4,2018-01-05,invierno,2018,ene,no festivo,vie,laborable,buen tiempo,9.305237,43.6957,12.522300,1518


In [8]:
numericas = df.select_dtypes(include = np.number).drop("usu_registrados", axis= 1)
numericas.head(2)

,temperatura,humedad,velo_viento
0,14.110847,80.5833,10.749882
1,14.902598,69.6087,16.652113


In [22]:
numericas_temp_sens = df_temp_sens.select_dtypes(include = np.number).drop("usu_registrados", axis= 1)
numericas_temp_sens.head(2)

,temperatura,sensacion_termica,humedad,velo_viento
0,14.110847,18.18125,80.5833,10.749882
1,14.902598,17.68695,69.6087,16.652113


In [9]:
robust = RobustScaler()
robust.fit(numericas)
X_robust = robust.transform(numericas)
numericas_robust = pd.DataFrame(X_robust, columns = numericas.columns)
numericas_robust.head(2)

,temperatura,humedad,velo_viento
0,-0.486274,0.858854,-0.208909
1,-0.425690,0.331548,0.687550


In [23]:
robust_temp_sens = RobustScaler()
robust_temp_sens.fit(numericas_temp_sens)
X_robust_temp_sens = robust_temp_sens.transform(numericas_temp_sens)
numericas_temp_sens_robust = pd.DataFrame(X_robust_temp_sens, columns = numericas_temp_sens.columns)
numericas_temp_sens_robust.head(2)

,temperatura,sensacion_termica,humedad,velo_viento
0,-0.486274,-0.456399,0.858854,-0.208909
1,-0.425690,-0.492863,0.331548,0.687550


In [10]:
df_robust = df.copy()
df_robust[numericas_robust.columns] = numericas_robust


In [24]:
df_robust_temp_sens = df_temp_sens.copy()
df_robust_temp_sens[numericas_temp_sens_robust.columns] = numericas_temp_sens_robust

In [11]:
df_robust.head(2)

,fecha,estacion,año,mes,festivo,dia,laborable,meteo,temperatura,humedad,velo_viento,usu_registrados
0,2018-01-01,invierno,2018,ene,festivo,lun,no laborable,nublado,-0.486274,0.858854,-0.208909,654
1,2018-01-02,invierno,2018,ene,no festivo,mar,laborable,nublado,-0.425690,0.331548,0.687550,670


In [25]:
df_robust_temp_sens.head(2)

,fecha,estacion,año,mes,festivo,dia,laborable,meteo,temperatura,sensacion_termica,humedad,velo_viento,usu_registrados
0,2018-01-01,invierno,2018,ene,festivo,lun,no laborable,nublado,-0.486274,-0.456399,0.858854,-0.208909,654
1,2018-01-02,invierno,2018,ene,no festivo,mar,laborable,nublado,-0.425690,-0.492863,0.331548,0.687550,670


In [26]:
# Guardamos el df normalizado:

df_robust.to_pickle('../data/bikes_registrados_estand.pkl')
df_robust_temp_sens.to_pickle('../data/bikes_registrados_estand_temp_sens.pkl')